In [3]:
import ee
import pandas as pd

In [4]:
# Initialize the Earth Engine module.
ee.Initialize()

In [8]:
PREFIX = 'asset-index'
FNAME = 'YOUR_FILENAME'
SCALE = 20  # meters
KSIZE = 112  # half the size of the desired patch in pixels (225 / 2)
BANDS = ['B2', 'B3', 'B4', 'B8', 'B11','B12']
SCALE_FACTOR = 10000  # Scaling factor for Sentinel-2 reflectance values
DATASET = 'COPERNICUS/S2'

def export_image(longtitude, latitude, start_date, end_date):

    # Define the point of interest and create a buffer region around it
    point = ee.Geometry.Point([longtitude, latitude])
    buffered_point = point.buffer(SCALE * KSIZE)  # Adjust buffer size as needed

    # Function to estimate cloud coverage over the ROI
    def cloudiness(image):
        qa_band = image.select('QA60')
        opaque_clouds = qa_band.bitwiseAnd(1 << 10).neq(0)
        cirrus_clouds = qa_band.bitwiseAnd(1 << 11).neq(0)
        total_cloud_mask = opaque_clouds.Or(cirrus_clouds)
        cloud_coverage_ratio = total_cloud_mask.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=buffered_point,
            scale=SCALE
        ).get('QA60')
        return image.set('cloudiness', cloud_coverage_ratio)

    # Load the Sentinel-2 image collection, filter, estimate cloudiness, and select least cloudy
    collection = ee.ImageCollection(DATASET) \
        .filterDate(start_date, end_date) \
        .filterBounds(buffered_point) \
        .map(cloudiness)

    # Select the least cloudy image
    least_cloudy_image = collection.sort('cloudiness').first()

    # Apply band selection and scaling to the chosen image
    processed_image = least_cloudy_image.select(BANDS).divide(SCALE_FACTOR)

    # Retrieve the cloudiness value and date of the image
    cloudiness_value = ee.Number(least_cloudy_image.get('cloudiness')).getInfo()
    date = ee.Date(least_cloudy_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

    # Define the point of interest as a feature collection
    points = ee.FeatureCollection([ee.Feature(point, {'longitude': longtitude, 'latitude': latitude})])

    # Set up the export task
    task = get_array_patches(
        processed_image,
        points,
        cloudiness_value,
        date,
        longtitude,
        latitude
    )

    # Start the export
    task.start()

# Function to get array patches
def get_array_patches(img, points, cloudiness_value, date, longitude, latitude):
    kern = ee.Kernel.square(radius=KSIZE, units='pixels')
    patches_array = img.neighborhoodToArray(kern)
    samples = points.map(lambda pt: sample_patch(pt, patches_array, cloudiness_value, date, longitude, latitude))

    # Export to TFRecord file
    task = ee.batch.Export.table.toDrive(
        collection=samples,
        description=PREFIX,
        folder=PREFIX,  
        fileNamePrefix=FNAME,
        fileFormat='TFRecord'
    )
    return task

# Function to sample patch and include additional properties
def sample_patch(point, patches_array, cloudiness_value, date, longitude, latitude):
    arrays_samples = patches_array.sample(
        region=point.geometry(),
        scale=SCALE,
        projection='EPSG:3857',
        factor=None,
        numPixels=None,
        dropNulls=False,
        tileScale=12)
    sample_feature = arrays_samples.first()
    # Set additional properties to the feature
    sample_feature = sample_feature.set({
        'cloudiness': cloudiness_value,
        'date': date,
        'longitude': longitude,
        'latitude': latitude
    })
    return sample_feature

In [9]:
df = pd.read_csv('../data/label.csv').head(1)

# Define your location and date range.
longtitude = df['centroid_longtitude'].values[0]
latitude   = df['centroid_latitude'].values[0]
start_date, end_date = '2020-12-01', '2020-12-31'

export_image(longtitude, latitude, start_date, end_date)